In [2]:
config_table_path = "abfss://enterpriseanalytics@fedsaentanalytics.dfs.core.windows.net/config/"
recreate = False

In [ ]:
# create ingestion config tables
if recreate and mssparkutils.fs.exists(f"{config_table_path}fe_config_ingest"):
	mssparkutils.fs.rm(f"{config_table_path}fe_config_ingest", True)

	if spark.catalog.tableExists("fe_config_ingest"):
		spark.sql("drop table fe_config_ingest")

if not spark.catalog.tableExists("fe_config_ingest"):
	sql = f"""CREATE TABLE fe_config_ingest
		    (
            uuid string 
            ,config_name string
            ,src_type string
            ,src_db_name string
            ,src_schema_name string
            ,src_table_name string
            ,src_col_def string
            ,src_query_init string
            ,src_query_inc string
            ,src_conn_secret_name string
            ,dest_delta_storage_account string
            ,dest_delta_table_container string
            ,temp_stage_container string
            ,mode string 
            ,inc_del_filter string 
            ,primary_key string
            ,partition_by string
            ,tag string
            ,src_file_container string
            ,src_file_directory string
            ,src_file_name string 
            ,excel_sheet_name string
            ,has_header boolean 
            ,rows_to_skip int 
            ,dw_server_name string
            ,dw_db_name string
            ,dw_schema_name string
            ,dw_table_name string
            ,is_disabled boolean
            ) USING DELTA
            LOCATION '{config_table_path}fe_config_ingest'"""

	print(sql)
	spark.sql(sql)

if recreate and mssparkutils.fs.exists(f"{config_table_path}fe_config_ingest_part"):
	mssparkutils.fs.rm(f"{config_table_path}fe_config_ingest_part", True)

	if spark.catalog.tableExists("fe_config_ingest_part"):
		spark.sql("drop table fe_config_ingest_part")

if not spark.catalog.tableExists("fe_config_ingest_part"):
	sql = f"""CREATE TABLE fe_config_ingest_part
		    (
             uuid string 
            ,az_id string
            ,part_filter string
            ,is_disabled boolean
            ) USING DELTA
            LOCATION '{config_table_path}fe_config_ingest_part'"""

	print(sql)
	spark.sql(sql)

In [11]:
# create ingestion views
sql = """
create or replace view vw_fe_config_ingest
		as
		select *
			, (select max(ts) from fe_config_ingest_log where az_id=x.az_id and op='dt_last_source_init') dt_last_source_init
			, (select max(ts) from fe_config_ingest_log where az_id=x.az_id and op='dt_last_source_inc') dt_last_source_inc
			, (select max(ts) from fe_config_ingest_log where az_id=x.az_id and op='dt_last_delta_init') dt_last_delta_init
			, (select max(ts) from fe_config_ingest_log where az_id=x.az_id and op='dt_last_delta_inc') dt_last_delta_inc
			, (select max(ts) from fe_config_ingest_log where az_id=x.az_id and op='dt_last_dw_load') dt_last_dw_load
		from (
				select replace(replace(replace(concat(config_name, '_', src_db_name,'_', src_schema_name,'_', src_table_name),'-',''),'/',''),'.','') as az_id
					,uuid, config_name, src_type, src_db_name, src_schema_name, src_table_name, src_col_def, src_conn_secret_name, dest_delta_storage_account, dest_delta_table_container
					,replace(replace(concat(config_name,'/',coalesce(src_db_name,''),'/',coalesce(src_schema_name,''),'/',replace(replace(src_table_name,'.','_'),',','_'),'/'),'//','/') ,'///','/') dest_delta_table_path
					,src_query_init, src_query_inc,primary_key,partition_by, inc_del_filter
					,mode, tag, is_disabled, dw_server_name, dw_db_name, dw_schema_name, dw_table_name
					,src_file_container ,src_file_directory ,src_file_name, excel_sheet_name, coalesce(has_header,cast(1 as boolean)) has_header, coalesce(rows_to_skip,0) rows_to_skip 
				from fe_config_ingest f
		) x;
"""

print(sql)
spark.sql(sql)


sql = """
create or replace view vw_fe_config_ingest_part
		as
			select *
			, (select max(ts) from fe_config_ingest_part_log where az_id=x.az_id and op='dt_part_created' and part_filter=x.part_filter) dt_part_created
			, (select max(ts) from fe_config_ingest_part_log where az_id=x.az_id and op='dt_part_last_seen' and part_filter=x.part_filter) dt_part_last_seen
			, (select max(ts) from fe_config_ingest_part_log where az_id=x.az_id and op='dt_part_last_load' and part_filter=x.part_filter) dt_part_last_load
			from (
						select c.*
							, 'select * from ('+src_query_init+')  src_x where '+ p.part_filter part_src_query_init
							, 'select * from ('+src_query_inc+')  src_x where '+ p.part_filter part_src_query_inc
							, p.uuid part_uuid, p.part_filter, p.is_disabled part_is_disabled
						from vw_fe_config_ingest c
							left join fe_config_ingest_part p on p.az_id=c.az_id
			) x;

"""

print(sql)
spark.sql(sql)

In [ ]:
# create transform config tables
if recreate and mssparkutils.fs.exists(f"{config_table_path}fe_config_transform"):
	mssparkutils.fs.rm(f"{config_table_path}fe_config_transform", True)

	if spark.catalog.tableExists("fe_config_transform"):
		spark.sql("drop table fe_config_transform")

if not spark.catalog.tableExists("fe_config_transform"):
	sql = f"""CREATE TABLE fe_config_transform
		(
		uuid string 
		,config_name string 
		,dest_delta_container string 
		,dest_delta_table_path string 
		,dest_delta_schema_name string 
		,dest_delta_table_name string 
		,dest_delta_table_primary_key string 
		,dest_delta_table_partition_by string 
		,dest_delta_table_z_order string 
		,sql_transform_init string 
		,sql_transform_inc string 
		,src_col_def string 
		,tag string 
		,mode string 
		,dest_dw_endpoint string 
		,dest_dw_db string 
		,is_disabled boolean 
		) USING DELTA
		LOCATION '{config_table_path}fe_config_transform'"""

	print(sql)
	spark.sql(sql)

if recreate and mssparkutils.fs.exists(f"{config_table_path}fe_config_transform_ref"):
	mssparkutils.fs.rm(f"{config_table_path}fe_config_transform_ref", True)

	if spark.catalog.tableExists("fe_config_transform_ref"):
		spark.sql("drop table fe_config_transform_ref")

if not spark.catalog.tableExists("fe_config_transform_ref"):
	sql = f"""CREATE TABLE fe_config_transform_ref
				(
				uuid string 
				,parent_az_id string 
				,az_id string
				,last_delta_version bigint 
				,is_disabled boolean 
				) USING DELTA
				LOCATION '{config_table_path}fe_config_transform_ref'"""

	print(sql)
	spark.sql(sql)

In [ ]:
# create transform views
sql = """
create or replace view vw_fe_config_transform 
		as
			select *
				, (select max(ts) from fe_config_transform_log l where az_id=x.az_id and l.op='dt_last_transform_init') dt_last_transform_init
				, (select max(ts) from fe_config_transform_log l where az_id=x.az_id and l.op='dt_last_transform_inc') dt_last_transform_inc
				, (select max(ts) from fe_config_transform_log l where az_id=x.az_id and l.op='dt_dw_last_transform_init') dt_dw_last_transform_init
                , (select max(ts) from fe_config_transform_log l where az_id=x.az_id and l.op='dt_dw_last_transform_inc') dt_dw_last_transform_inc 
			from (
				select replace(replace(concat(config_name, '_', dest_delta_container,'_', dest_delta_schema_name,'_', dest_delta_table_name),'-',''),'/','') as az_id
					,uuid, config_name, dest_delta_container, dest_delta_table_path, dest_delta_schema_name, dest_delta_table_name
					, dest_delta_table_primary_key, dest_delta_table_partition_by, dest_delta_table_z_order
					, sql_transform_init, sql_transform_inc, tag, mode
					, src_col_def, is_disabled, dest_dw_endpoint, dest_dw_db
				from fe_config_transform f
			) x
			
"""

print(sql)
spark.sql(sql)


sql = """
create or replace view vw_fe_config_transform_ref
		as
			select r.parent_az_id parent_id, t.az_id, t.config_name, t.tag, r.last_delta_version
				, coalesce(t_parent.dt_last_transform_inc, t_parent.dt_last_transform_init, i_parent2.dt_last_delta_inc, i_parent2.dt_last_delta_init, t_parent.dt_dw_last_transform_inc, t_parent.dt_dw_last_transform_init) dt_last_parent_load
				, coalesce(t_parent.dest_delta_container, i_parent2.dest_delta_table_container) parent_table_container
				, coalesce(t_parent.dest_delta_table_path, i_parent2.dest_delta_table_path) parent_table_path
				, i_parent.parent_az_id parent_full_delta_path
				, r.is_disabled is_disabled, t.dest_dw_endpoint, t.dest_dw_db 
				from vw_fe_config_transform t
						left join fe_config_transform_ref r on r.az_id=t.az_id
						left join vw_fe_config_transform t_parent on t_parent.az_id=r.parent_az_id	
						left join lateral (select x.*
									from fe_config_transform_ref x
									where left(x.parent_az_id,8)='abfss://'
										and x.az_id=r.parent_az_id
									) i_parent
                        left join vw_fe_config_ingest i_parent2 on i_parent2.az_id=r.parent_az_id
"""

print(sql)
spark.sql(sql)

In [ ]:
# create ingest log tables
if recreate and mssparkutils.fs.exists(f"{config_table_path}fe_config_ingest_log"):
	mssparkutils.fs.rm(f"{config_table_path}fe_config_ingest_log", True)

	if spark.catalog.tableExists("fe_config_ingest_log"):
		spark.sql("drop table fe_config_ingest_log")

if not spark.catalog.tableExists("fe_config_ingest_log"):
	sql = f"""CREATE TABLE fe_config_ingest_log
		(
		uuid string 
		,az_id string 
		,op string 
		,ts timestamp
        ,msg string 
		) USING DELTA
		LOCATION '{config_table_path}fe_config_ingest_log'
        PARTITIONED BY (az_id)"""

	print(sql)
	spark.sql(sql)

if recreate and mssparkutils.fs.exists(f"{config_table_path}fe_config_ingest_part_log"):
	mssparkutils.fs.rm(f"{config_table_path}fe_config_ingest_part_log", True)

	if spark.catalog.tableExists("fe_config_ingest_part_log"):
		spark.sql("drop table fe_config_ingest_part_log")

if not spark.catalog.tableExists("fe_config_ingest_part_log"):
	sql = f"""CREATE TABLE fe_config_ingest_part_log
				(
				uuid string 
                ,az_id string 
                ,part_filter string
                ,op string 
                ,ts timestamp
                ,msg string 
				) USING DELTA
				LOCATION '{config_table_path}fe_config_ingest_part_log'
                PARTITIONED BY (az_id)"""

	print(sql)
	spark.sql(sql)

In [ ]:
# create transform log tables
if recreate and mssparkutils.fs.exists(f"{config_table_path}fe_config_transform_log"):
	mssparkutils.fs.rm(f"{config_table_path}fe_config_transform_log", True)

	if spark.catalog.tableExists("fe_config_transform_log"):
		spark.sql("drop table fe_config_transform_log")

if not spark.catalog.tableExists("fe_config_transform_log"):
	sql = f"""CREATE TABLE fe_config_transform_log
		(
		uuid string 
		,az_id string 
		,op string 
		,ts timestamp
        ,msg string 
		) USING DELTA
		LOCATION '{config_table_path}fe_config_transform_log'
        PARTITIONED BY (az_id)"""

	print(sql)
	spark.sql(sql)

In [5]:
%run nb_framework_common

In [10]:
# create equivelent views in serverless... above spark delta tables get put in catalog by default in lake database... views dont... so will unfortunately have to make sure def matches between two as changes are made
config_db = "fe_config"        
cnnstr,token=get_serverless_synapse_conn()

# create sql db for adls container ... aka bronze,silver,gold
run_serverless_sql(f"IF NOT EXISTS (SELECT * FROM master.sys.databases WHERE name = '{config_db}') BEGIN CREATE DATABASE [{config_db}] END", cnnstr, token)
run_serverless_sql(f"alter database {config_db} COLLATE SQL_Latin1_General_CP1_CI_AS", cnnstr, token)

#switch db to config db
cnnstr,token=get_serverless_synapse_conn(dbname=config_db)

run_serverless_sql(f"""
            IF (EXISTS (select * from information_schema.tables where table_schema = 'dbo' and table_name='fe_config_ingest')) 
            BEGIN
                EXEC('DROP VIEW [dbo].[fe_config_ingest]')
            END
            """, cnnstr, token)

#create table views
create_view_statement = f"""
                create view [dbo].[fe_config_ingest] as
                    select * 
                    from openrowset(bulk '{config_table_path}fe_config_ingest', format='DELTA')
                    with ( [uuid] uniqueidentifier 
                            ,[config_name] varchar(128) COLLATE SQL_Latin1_General_CP1_CI_AS
                            ,[src_type] varchar(128) COLLATE SQL_Latin1_General_CP1_CI_AS
                            ,[src_db_name] varchar(128) COLLATE SQL_Latin1_General_CP1_CI_AS
                            ,[src_schema_name] varchar(128) COLLATE SQL_Latin1_General_CP1_CI_AS
                            ,[src_table_name] varchar(128) COLLATE SQL_Latin1_General_CP1_CI_AS
                            ,[src_col_def] varchar(max) COLLATE SQL_Latin1_General_CP1_CI_AS
                            ,[src_query_init] varchar(max) COLLATE SQL_Latin1_General_CP1_CI_AS
                            ,[src_query_inc] varchar(max) COLLATE SQL_Latin1_General_CP1_CI_AS
                            ,[src_conn_secret_name] varchar(128) COLLATE SQL_Latin1_General_CP1_CI_AS
                            ,[dest_delta_table_container] varchar(128) COLLATE SQL_Latin1_General_CP1_CI_AS
                            ,[temp_stage_container] varchar(128) COLLATE SQL_Latin1_General_CP1_CI_AS
                            ,[mode] varchar(128) COLLATE SQL_Latin1_General_CP1_CI_AS
                            ,[inc_del_filter] varchar(128) COLLATE SQL_Latin1_General_CP1_CI_AS
                            ,[primary_key] varchar(1024) COLLATE SQL_Latin1_General_CP1_CI_AS
                            ,[partition_by] varchar(1024) COLLATE SQL_Latin1_General_CP1_CI_AS
                            ,[tag] varchar(128) COLLATE SQL_Latin1_General_CP1_CI_AS
                            ,[src_file_container] varchar(256) COLLATE SQL_Latin1_General_CP1_CI_AS
                            ,[src_file_directory] varchar(256) COLLATE SQL_Latin1_General_CP1_CI_AS
                            ,[src_file_name] varchar(128) COLLATE SQL_Latin1_General_CP1_CI_AS
                            ,[excel_sheet_name] varchar(128) COLLATE SQL_Latin1_General_CP1_CI_AS
                            ,[has_header] bit 
                            ,[rows_to_skip] int 
                            ,[is_disabled] bit
                            ,[dw_schema_name] varchar(128) COLLATE SQL_Latin1_General_CP1_CI_AS
                            ,[dw_table_name] varchar(128) COLLATE SQL_Latin1_General_CP1_CI_AS
                            ,[dest_delta_storage_account] varchar(128) COLLATE SQL_Latin1_General_CP1_CI_AS
                            ,[dw_server_name] varchar(128) COLLATE SQL_Latin1_General_CP1_CI_AS
                            ,[dw_db_name] varchar(128) COLLATE SQL_Latin1_General_CP1_CI_AS) as v
"""
run_serverless_sql(create_view_statement, cnnstr, token)

run_serverless_sql(f"""
            IF (EXISTS (select * from information_schema.tables where table_schema = 'dbo' and table_name='fe_config_ingest_part')) 
            BEGIN
                EXEC('DROP VIEW [dbo].[fe_config_ingest_part]')
            END
            """, cnnstr, token)

create_view_statement = f"""
                create view [dbo].[fe_config_ingest_part] as
                    select * 
                    from openrowset(bulk '{config_table_path}fe_config_ingest_part', format='DELTA')
                    with (
                         uuid uniqueidentifier 
                        ,[az_id] varchar(128) COLLATE SQL_Latin1_General_CP1_CI_AS
                        ,part_filter varchar(2048) COLLATE SQL_Latin1_General_CP1_CI_AS
                        ,is_disabled bit
                    ) as v
"""
run_serverless_sql(create_view_statement, cnnstr, token)

run_serverless_sql(f"""
            IF (EXISTS (select * from information_schema.tables where table_schema = 'dbo' and table_name='fe_config_transform')) 
            BEGIN
                EXEC('DROP VIEW [dbo].[fe_config_transform]')
            END
            """, cnnstr, token)

create_view_statement = f"""
                create view [dbo].[fe_config_transform] as
                    select * 
                    from openrowset(bulk '{config_table_path}fe_config_transform', format='DELTA') 
                    with (
                         uuid uniqueidentifier 
                        ,[config_name] varchar(128) COLLATE SQL_Latin1_General_CP1_CI_AS
                        ,[dest_delta_container] varchar(128) COLLATE SQL_Latin1_General_CP1_CI_AS
                        ,[dest_delta_table_path] varchar(1024) COLLATE SQL_Latin1_General_CP1_CI_AS
                        ,[dest_delta_schema_name] varchar(1024) COLLATE SQL_Latin1_General_CP1_CI_AS
                        ,[dest_delta_table_name] varchar(128) COLLATE SQL_Latin1_General_CP1_CI_AS
                        ,[dest_delta_table_primary_key] varchar(512) COLLATE SQL_Latin1_General_CP1_CI_AS
                        ,[dest_delta_table_partition_by] varchar(512) COLLATE SQL_Latin1_General_CP1_CI_AS
                        ,[dest_delta_table_z_order] varchar(512) COLLATE SQL_Latin1_General_CP1_CI_AS
                        ,[sql_transform_init] varchar(max) COLLATE SQL_Latin1_General_CP1_CI_AS
                        ,[sql_transform_inc] varchar(max) COLLATE SQL_Latin1_General_CP1_CI_AS
                        ,src_col_def varchar(max) COLLATE SQL_Latin1_General_CP1_CI_AS
                        ,[tag] varchar(128) COLLATE SQL_Latin1_General_CP1_CI_AS
                        ,[refs] varchar(max) COLLATE SQL_Latin1_General_CP1_CI_AS
                        ,[mode] varchar(128) COLLATE SQL_Latin1_General_CP1_CI_AS
                        ,[dest_dw_endpoint] varchar(128) COLLATE SQL_Latin1_General_CP1_CI_AS
                        ,[dest_dw_db] varchar(128) COLLATE SQL_Latin1_General_CP1_CI_AS
                        ,is_disabled bit 
                    ) as v
"""
run_serverless_sql(create_view_statement, cnnstr, token)

run_serverless_sql(f"""
            IF (EXISTS (select * from information_schema.tables where table_schema = 'dbo' and table_name='fe_config_transform_ref')) 
            BEGIN
                EXEC('DROP VIEW [dbo].[fe_config_transform_ref]')
            END
            """, cnnstr, token)

create_view_statement = f"""
                create view [dbo].[fe_config_transform_ref] as
                    select * 
                    from openrowset(bulk '{config_table_path}fe_config_transform_ref', format='DELTA') 
                    with (
                        uuid uniqueidentifier 
                        ,[parent_az_id] varchar(128) COLLATE SQL_Latin1_General_CP1_CI_AS
                        ,[az_id] varchar(128) COLLATE SQL_Latin1_General_CP1_CI_AS
                        ,last_delta_version bigint 
                        ,is_disabled bit 
                    )as v
"""
run_serverless_sql(create_view_statement, cnnstr, token)

run_serverless_sql(f"""
            IF (EXISTS (select * from information_schema.tables where table_schema = 'dbo' and table_name='fe_config_ingest_log')) 
            BEGIN
                EXEC('DROP VIEW [dbo].[fe_config_ingest_log]')
            END
            """, cnnstr, token)

create_view_statement = f"""
                create view [dbo].[fe_config_ingest_log] as
                    select * 
                    from openrowset(bulk '{config_table_path}fe_config_ingest_log', format='DELTA')
                    with ( [uuid] uniqueidentifier 
                            ,[az_id] varchar(1024) COLLATE SQL_Latin1_General_CP1_CI_AS
                            ,[op] varchar(128) COLLATE SQL_Latin1_General_CP1_CI_AS
                            ,[ts] datetime2 
                            ,[msg] varchar(max) COLLATE SQL_Latin1_General_CP1_CI_AS
                        ) as v
"""
run_serverless_sql(create_view_statement, cnnstr, token)

run_serverless_sql(f"""
            IF (EXISTS (select * from information_schema.tables where table_schema = 'dbo' and table_name='fe_config_ingest_part_log')) 
            BEGIN
                EXEC('DROP VIEW [dbo].[fe_config_ingest_part_log]')
            END
            """, cnnstr, token)

#create table views
create_view_statement = f"""
                create view [dbo].[fe_config_ingest_part_log] as
                    select * 
                    from openrowset(bulk '{config_table_path}fe_config_ingest_part_log', format='DELTA')
                    with ( [uuid] uniqueidentifier 
                            ,[az_id] varchar(1024) COLLATE SQL_Latin1_General_CP1_CI_AS
                            ,[part_filter] varchar(2048) COLLATE SQL_Latin1_General_CP1_CI_AS
                            ,[op] varchar(128) COLLATE SQL_Latin1_General_CP1_CI_AS
                            ,[ts] datetime2 
                            ,[msg] varchar(max) COLLATE SQL_Latin1_General_CP1_CI_AS
                        ) as v
"""
run_serverless_sql(create_view_statement, cnnstr, token) 

run_serverless_sql(f"""
            IF (EXISTS (select * from information_schema.tables where table_schema = 'dbo' and table_name='fe_config_transform_log')) 
            BEGIN
                EXEC('DROP VIEW [dbo].[fe_config_transform_log]')
            END
            """, cnnstr, token)

create_view_statement = f"""
                create view [dbo].[fe_config_transform_log] as
                    select * 
                    from openrowset(bulk '{config_table_path}fe_config_transform_log', format='DELTA')
                    with ( [uuid] uniqueidentifier 
                            ,[az_id] varchar(1024) COLLATE SQL_Latin1_General_CP1_CI_AS
                            ,[op] varchar(128) COLLATE SQL_Latin1_General_CP1_CI_AS
                            ,[ts] datetime2 
                            ,[msg] varchar(max) COLLATE SQL_Latin1_General_CP1_CI_AS
                        ) as v
"""
run_serverless_sql(create_view_statement, cnnstr, token)

#create consumption views
run_serverless_sql(f"""
            IF (EXISTS (select * from information_schema.tables where table_schema = 'dbo' and table_name='vw_fe_config_ingest')) 
            BEGIN
                EXEC('DROP VIEW [dbo].[vw_fe_config_ingest]')
            END
            """, cnnstr, token)

create_view_statement = f"""
        create view dbo.vw_fe_config_ingest
		as
		select *
			, (select max(ts) from fe_config_ingest_log where az_id=x.az_id and op='dt_last_source_init') dt_last_source_init
			, (select max(ts) from fe_config_ingest_log where az_id=x.az_id and op='dt_last_source_inc') dt_last_source_inc
			, (select max(ts) from fe_config_ingest_log where az_id=x.az_id and op='dt_last_delta_init') dt_last_delta_init
			, (select max(ts) from fe_config_ingest_log where az_id=x.az_id and op='dt_last_delta_inc') dt_last_delta_inc
            , (select max(ts) from fe_config_ingest_log where az_id=x.az_id and op='dt_last_dw_load') dt_last_dw_load
		from (
				select replace(replace(replace(concat(config_name, '_', src_db_name,'_', src_schema_name,'_', src_table_name),'-',''),'/',''),'.','') as az_id
					,uuid, config_name, src_type, src_db_name, src_schema_name, src_table_name, src_col_def, src_conn_secret_name, dest_delta_storage_account, dest_delta_table_container
					,src_query_init, src_query_inc,primary_key,partition_by, inc_del_filter
					, replace(replace(concat(config_name,'/',coalesce(src_db_name,''),'/',coalesce(src_schema_name,''),'/',replace(replace(src_table_name,'.','_'),',','_'),'/'),'//','/') ,'///','/') [dest_delta_table_path]
                    , mode, tag, is_disabled,src_file_container ,src_file_directory ,src_file_name, excel_sheet_name, coalesce(has_header,cast(1 as bit)) has_header, coalesce(rows_to_skip,0) rows_to_skip 
                    , dw_server_name, dw_db_name,dw_schema_name, dw_table_name
                from fe_config_ingest f
		) x
"""
run_serverless_sql(create_view_statement, cnnstr, token)

run_serverless_sql(f"""
            IF (EXISTS (select * from information_schema.tables where table_schema = 'dbo' and table_name='vw_fe_config_ingest_part')) 
            BEGIN
                EXEC('DROP VIEW [dbo].[vw_fe_config_ingest_part]')
            END
            """, cnnstr, token)

create_view_statement = f"""
        create view dbo.vw_fe_config_ingest_part
		as
			select *
			, (select max(ts) from fe_config_ingest_part_log where az_id=x.az_id and op='dt_part_created' and part_filter=x.part_filter) dt_part_created
			, (select max(ts) from fe_config_ingest_part_log where az_id=x.az_id and op='dt_part_last_seen' and part_filter=x.part_filter) dt_part_last_seen
			, (select max(ts) from fe_config_ingest_part_log where az_id=x.az_id and op='dt_part_last_load' and part_filter=x.part_filter) dt_part_last_load
			from (
						select c.*
							, 'select * from ('+src_query_init+')  src_x where '+ p.part_filter part_src_query_init
							, 'select * from ('+src_query_inc+')  src_x where '+ p.part_filter part_src_query_inc
							, p.uuid part_uuid, part_filter, p.is_disabled part_is_disabled
						from dbo.vw_fe_config_ingest c
							inner join dbo.fe_config_ingest_part p on p.az_id=c.az_id
			) x
"""
run_serverless_sql(create_view_statement, cnnstr, token)

run_serverless_sql(f"""
            IF (EXISTS (select * from information_schema.tables where table_schema = 'dbo' and table_name='vw_fe_config_transform')) 
            BEGIN
                EXEC('DROP VIEW [dbo].[vw_fe_config_transform]')
            END
            """, cnnstr, token)

create_view_statement = f"""
        create view dbo.vw_fe_config_transform
		as
            select *
                , (select max(ts) from fe_config_transform_log l where l.az_id=x.az_id and l.op='dt_last_transform_init') dt_last_transform_init
                , (select max(ts) from fe_config_transform_log l where l.az_id=x.az_id and l.op='dt_last_transform_inc') dt_last_transform_inc 
                , (select max(ts) from fe_config_transform_log l where l.az_id=x.az_id and l.op='dt_dw_last_transform_init') dt_dw_last_transform_init
                , (select max(ts) from fe_config_transform_log l where l.az_id=x.az_id and l.op='dt_dw_last_transform_inc') dt_dw_last_transform_inc 
            from (
                select replace(replace(concat(config_name, '_', dest_delta_container,'_', dest_delta_schema_name,'_', dest_delta_table_name),'-',''),'/','') as az_id
                ,uuid, config_name, dest_delta_container, dest_delta_table_path, dest_delta_schema_name, dest_delta_table_name
                , dest_delta_table_primary_key, dest_delta_table_partition_by, dest_delta_table_z_order
                , sql_transform_init, sql_transform_inc, tag, mode
                , src_col_def, is_disabled, dest_dw_endpoint, dest_dw_db
                from fe_config_transform f
            ) x
			
"""
run_serverless_sql(create_view_statement, cnnstr, token)

run_serverless_sql(f"""
            IF (EXISTS (select * from information_schema.tables where table_schema = 'dbo' and table_name='vw_fe_config_transform_ref')) 
            BEGIN
                EXEC('DROP VIEW [dbo].[vw_fe_config_transform_ref]')
            END
            """, cnnstr, token)

create_view_statement = f"""
        create view dbo.vw_fe_config_transform_ref
		as
			select r.parent_az_id parent_id, t.az_id, t.config_name, t.tag, r.last_delta_version
				, coalesce(t_parent.dt_last_transform_inc, t_parent.dt_last_transform_init, i_parent2.dt_last_delta_inc, i_parent2.dt_last_delta_init, t_parent.dt_dw_last_transform_inc, t_parent.dt_dw_last_transform_init) dt_last_parent_load
				, coalesce(t_parent.dest_delta_container, i_parent2.dest_delta_table_container) parent_table_container
				, coalesce(t_parent.dest_delta_table_path, i_parent2.dest_delta_table_path) parent_table_path
				, i_parent.parent_az_id parent_full_delta_path
				, r.is_disabled is_disabled, t.dest_dw_endpoint, t.dest_dw_db 
				from vw_fe_config_transform t
						left join fe_config_transform_ref r on r.az_id=t.az_id
						left join vw_fe_config_transform t_parent on t_parent.az_id=r.parent_az_id	
						outer apply (select x.*
									from fe_config_transform_ref x
									where left(x.parent_az_id,8)='abfss://'
										and x.az_id=r.parent_az_id
									) i_parent
                        left join vw_fe_config_ingest i_parent2 on i_parent2.az_id=r.parent_az_id
"""
run_serverless_sql(create_view_statement, cnnstr, token)



 